In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import argparse
import pickle 
from src.utils import ReaderWriter, create_directory
from src.utils import one_hot_encode
from src.utils import get_char, outer_cross_val
import itertools
from sklearn.model_selection import KFold
import xgboost as xgb
from src.utils import ReaderWriter, create_directory, perfmetric_report_cont
from src.utils import compute_spearman_corr, compute_pearson_corr
import matplotlib.pyplot as plt
from src.utils import performance_statistics,seq_to_onehot

In [2]:
#. Reading the data
data_name = '20240115_for_ML_mismatch_model_potential.xlsx'
data_dir = './data/'
df = pd.read_excel(os.path.join(data_dir,data_name ) )
df

,NAME,ORIGINAL_SEQ,MISMATCHED_SEQ,EFFICIENCY,MISMATCH_TYPE,POSITION_FROM_LEFT
0,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,ATCAGTGTGATAGGATCCGT,2.801900,G->A,1
1,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GCCAGTGTGATAGGATCCGT,6.401032,T->C,2
2,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTTAGTGTGATAGGATCCGT,3.629865,C->T,3
3,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCGGTGTGATAGGATCCGT,1.973759,A->G,4
4,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCAATGTGATAGGATCCGT,8.968154,G->A,5
...,...,...,...,...,...,...
311,2MM_rev_oRNA_HEKs_18_2_human,GAACGTCTGTCCTTACCCCT,GAACGTCTGTCCTTACCCGA,48.705436,"C->G, T->A","19, 20"
312,0MM_non_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCAGTGTGATAGGATCCGT,62.675635,no_MM,no_MM
313,0MM_non_oRNA_DNMT1_1050_mouse,GTAGTCAGAATACTTGCGGT,GTAGTCAGAATACTTGCGGT,43.151624,no_MM,no_MM
314,0MM_non_oRNA_EMX1_human,GTGATGGGAGCCCTTCTTCT,GTGATGGGAGCCCTTCTTCT,31.895831,no_MM,no_MM


In [3]:
### remove the nan
df = df.dropna()
df = df.reset_index(drop=True)
df

,NAME,ORIGINAL_SEQ,MISMATCHED_SEQ,EFFICIENCY,MISMATCH_TYPE,POSITION_FROM_LEFT
0,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,ATCAGTGTGATAGGATCCGT,2.801900,G->A,1
1,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GCCAGTGTGATAGGATCCGT,6.401032,T->C,2
2,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTTAGTGTGATAGGATCCGT,3.629865,C->T,3
3,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCGGTGTGATAGGATCCGT,1.973759,A->G,4
4,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCAATGTGATAGGATCCGT,8.968154,G->A,5
...,...,...,...,...,...,...
309,2MM_rev_oRNA_HEKs_18_2_human,GAACGTCTGTCCTTACCCCT,GAACGTCTGTCCTTACCCGA,48.705436,"C->G, T->A","19, 20"
310,0MM_non_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCAGTGTGATAGGATCCGT,62.675635,no_MM,no_MM
311,0MM_non_oRNA_DNMT1_1050_mouse,GTAGTCAGAATACTTGCGGT,GTAGTCAGAATACTTGCGGT,43.151624,no_MM,no_MM
312,0MM_non_oRNA_EMX1_human,GTGATGGGAGCCCTTCTTCT,GTGATGGGAGCCCTTCTTCT,31.895831,no_MM,no_MM


In [4]:
##2.prepare the data
unique_seq = df['ORIGINAL_SEQ'].apply(get_char)
missmatch_seq = df['MISMATCHED_SEQ'].apply(get_char)
y = np.array(df['EFFICIENCY'])
onehot_x_1 = seq_to_onehot(unique_seq)
onehot_x_2 = seq_to_onehot(missmatch_seq)
onehot_x_1.shape
x = np.concatenate([onehot_x_1, onehot_x_2], axis=1)
x.shape

(314, 160)

In [ ]:
## 3. train by splitting the data according to the pairs (original and missmatched sequence)
#output_path = './miss_match/standard_data_split'
output_path = './miss_match'
data_partitions = outer_cross_val(onehot_x_1.shape[0], 5)

In [6]:
other_params = {'subsample': 1.0,'min_child_weight': 10, 'gamma': 2.5,'colsample_bytree': 0.4,'alpha': 0, 'learning_rate':0.1}
param_vals = {'n_estimators': [600],'max_depth': [12], 'reg_lambda': [1, 10, 100]}
N_innerCV  =2
inner_cv = KFold(n_splits=N_innerCV, shuffle=True, random_state=0)

In [ ]:
## 4.train
hyper_param = param_vals.values()
combinations = list(itertools.product(*hyper_param))
test_error = []
test_spearm_corr = []
test_pearson_corr = []
# outer cross validation
for i in range(len(data_partitions)):
    print(f"Outer Fold {i}:")
    print(f"   number of Training instances:", len(data_partitions[i]['train_index']))
    print(f"   number of test instances:", len(data_partitions[i]['test_index']))
    error = np.empty([len(data_partitions),len(combinations)])
    ## inner cross validation for hyper param tuning
    for j, (train_index_new, val_index) in enumerate(inner_cv.split(data_partitions[i]['train_index'])):
        print(f"Inner Fold {j}:")
        print(f"   number of Training instances:", len(train_index_new))
        print(f"   number of validation instances:", len(val_index))
    
        for index, c in enumerate(combinations):
            model = xgb.XGBRegressor(n_estimators=c[0], max_depth=c[1], reg_lambda=c[2], **other_params)
            model.fit(x[train_index_new,:], y[train_index_new], eval_set = [(x[val_index,:], y[val_index])], early_stopping_rounds=10) 
            ## train error:
            yhat_train = model.predict(x[train_index_new,:])
            train_error = np.absolute(y[train_index_new]-yhat_train).mean()
            #print('training error:', train_error)
            yhat_val = model.predict(x[val_index,:])
            #error = MAE(y[val_index], yhat)
            val_error = np.absolute(y[val_index]-yhat_val).mean()
            #print('validation error:', val_error)
            error[j,index] =val_error
            
    #get the best paramerter values       
    best_param_indx = error.mean(axis = 0).argmin()
    best_param = combinations[best_param_indx]
    #best_param = [100, 12, 10]
    print('best parameters:', best_param)
    #train the model with the best parameters
    model = xgb.XGBRegressor(n_estimators=best_param[0], max_depth=best_param[1], reg_lambda=best_param[2], **other_params)
    #predict on the test set 
    model.fit(x[data_partitions[i]['train_index'],:], y[data_partitions[i]['train_index']])
    yhat_test = model.predict(x[data_partitions[i]['test_index'],:])
    t_error = np.absolute(y[data_partitions[i]['test_index']]-yhat_test).mean()
    ## creat directory to save the model
    path_to_save = create_directory('run_'+ str(i), output_path)
    ## save the best model for this run
    model.save_model(path_to_save +'/best.model')
    ReaderWriter.write_csv(path_to_save+'/prediction', y[data_partitions[i]['test_index']],yhat_test, header=['true','predicted'])
    #ReaderWriter.write_log()
    test_error.append(t_error)
    test_spearm_corr.append(compute_spearman_corr(yhat_test,y[data_partitions[i]['test_index']]))
    test_pearson_corr.append(compute_pearson_corr(yhat_test,y[data_partitions[i]['test_index']]))
    perfmetric_report_cont(yhat_test, y[data_partitions[i]['test_index']], path_to_save +'/test_performance.csv' )
    ReaderWriter.write_params(path_to_save,param_vals,  best_param, other_params)
    fig, ax = plt.subplots()
    ax.scatter(yhat_test,y[data_partitions[i]['test_index']],  vmin=0, vmax=100)

performance_statistics(len(data_partitions), output_path)



In [8]:
performance_statistics(len(data_partitions), output_path)



Spearman Correlation: Mean = 0.7333764011302861 , Std = 0.07349276836001198
Spearman P-Value: Mean = 1.8774856923675426e-09 , Std = 3.350146058360265e-09
Pearson Correlation: Mean = 0.9078966800158655 , Std = 0.01793635094431689
Pearson P-Value: Mean = 2.1671487893712296e-22 , Std = 3.7807270899142507e-22
MAE Score: Mean = 9.437229745743858 , Std = 1.3841802206945841


## Train the model with leave one out split strategy

In [16]:
### Train the model with leave one out split strategy where we leave a target site and all its miss matched pairs
## from the training and test on the new target site and its missmatched pairs( testing the model on totally unseen target)
#. Reading the data
data_name = '20240115_for_ML_mismatch_model_potential.xlsx'
data_dir = './data/'
df = pd.read_excel(os.path.join(data_dir,data_name ) )
### remove the nan
df = df.dropna()
df = df.reset_index(drop=True)
df

,NAME,ORIGINAL_SEQ,MISMATCHED_SEQ,EFFICIENCY,MISMATCH_TYPE,POSITION_FROM_LEFT
0,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,ATCAGTGTGATAGGATCCGT,2.801900,G->A,1
1,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GCCAGTGTGATAGGATCCGT,6.401032,T->C,2
2,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTTAGTGTGATAGGATCCGT,3.629865,C->T,3
3,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCGGTGTGATAGGATCCGT,1.973759,A->G,4
4,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCAATGTGATAGGATCCGT,8.968154,G->A,5
...,...,...,...,...,...,...
309,2MM_rev_oRNA_HEKs_18_2_human,GAACGTCTGTCCTTACCCCT,GAACGTCTGTCCTTACCCGA,48.705436,"C->G, T->A","19, 20"
310,0MM_non_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCAGTGTGATAGGATCCGT,62.675635,no_MM,no_MM
311,0MM_non_oRNA_DNMT1_1050_mouse,GTAGTCAGAATACTTGCGGT,GTAGTCAGAATACTTGCGGT,43.151624,no_MM,no_MM
312,0MM_non_oRNA_EMX1_human,GTGATGGGAGCCCTTCTTCT,GTGATGGGAGCCCTTCTTCT,31.895831,no_MM,no_MM


In [17]:
df['ORIGINAL_SEQ'].unique()

array(['GTCAGTGTGATAGGATCCGT', 'GTAGTCAGAATACTTGCGGT',
       'GTGATGGGAGCCCTTCTTCT', 'GAACGTCTGTCCTTACCCCT'], dtype=object)

In [18]:
# Assuming you have already imported pandas as pd and your DataFrame is named df
test_seq = 'GTCAGTGTGATAGGATCCGT'
train_df = df[~df['ORIGINAL_SEQ'].str.contains(test_seq)]
test_df =  df[df['ORIGINAL_SEQ'].str.contains(test_seq)]

In [12]:
train_df

,NAME,ORIGINAL_SEQ,MISMATCHED_SEQ,EFFICIENCY,MISMATCH_TYPE,POSITION_FROM_LEFT
20,1MM_pur_oRNA_DNMT1_1050_mouse,GTAGTCAGAATACTTGCGGT,ATAGTCAGAATACTTGCGGT,3.957317,G->A,1
21,1MM_pur_oRNA_DNMT1_1050_mouse,GTAGTCAGAATACTTGCGGT,GCAGTCAGAATACTTGCGGT,2.459427,T->C,2
22,1MM_pur_oRNA_DNMT1_1050_mouse,GTAGTCAGAATACTTGCGGT,GTGGTCAGAATACTTGCGGT,0.844803,A->G,3
23,1MM_pur_oRNA_DNMT1_1050_mouse,GTAGTCAGAATACTTGCGGT,GTAATCAGAATACTTGCGGT,8.346463,G->A,4
24,1MM_pur_oRNA_DNMT1_1050_mouse,GTAGTCAGAATACTTGCGGT,GTAGCCAGAATACTTGCGGT,7.354122,T->C,5
...,...,...,...,...,...,...
308,2MM_rev_oRNA_HEKs_18_2_human,GAACGTCTGTCCTTACCCCT,GAACGTCTGTCCTTACCGGT,41.702427,"C->G, C->G","18, 19"
309,2MM_rev_oRNA_HEKs_18_2_human,GAACGTCTGTCCTTACCCCT,GAACGTCTGTCCTTACCCGA,48.705436,"C->G, T->A","19, 20"
311,0MM_non_oRNA_DNMT1_1050_mouse,GTAGTCAGAATACTTGCGGT,GTAGTCAGAATACTTGCGGT,43.151624,no_MM,no_MM
312,0MM_non_oRNA_EMX1_human,GTGATGGGAGCCCTTCTTCT,GTGATGGGAGCCCTTCTTCT,31.895831,no_MM,no_MM


In [13]:
test_df

,NAME,ORIGINAL_SEQ,MISMATCHED_SEQ,EFFICIENCY,MISMATCH_TYPE,POSITION_FROM_LEFT
0,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,ATCAGTGTGATAGGATCCGT,2.801900,G->A,1
1,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GCCAGTGTGATAGGATCCGT,6.401032,T->C,2
2,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTTAGTGTGATAGGATCCGT,3.629865,C->T,3
3,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCGGTGTGATAGGATCCGT,1.973759,A->G,4
4,1MM_pur_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCAATGTGATAGGATCCGT,8.968154,G->A,5
...,...,...,...,...,...,...
251,2MM_rev_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCAGTGTGATAGGAAGCGT,74.963963,"T->A, C->G","16, 17"
252,2MM_rev_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCAGTGTGATAGGATGGGT,80.524939,"C->G, C->G","17, 18"
253,2MM_rev_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCAGTGTGATAGGATCGCT,77.191697,"C->G, G->C","18, 19"
254,2MM_rev_oRNA_DNMT1_1054_mouse,GTCAGTGTGATAGGATCCGT,GTCAGTGTGATAGGATCCCA,85.023695,"G->C, T->A","19, 20"


In [14]:
##2.prepare the data
unique_seq = train_df['ORIGINAL_SEQ'].apply(get_char)
missmatch_seq = train_df['MISMATCHED_SEQ'].apply(get_char)
train_y = np.array(train_df['EFFICIENCY'])
onehot_x_1 = seq_to_onehot(unique_seq)
onehot_x_2 = seq_to_onehot(missmatch_seq)
onehot_x_1.shape
train_x = np.concatenate([onehot_x_1, onehot_x_2], axis=1)
train_x.shape

(235, 160)

In [15]:
### test:
unique_seq_test = test_df['ORIGINAL_SEQ'].apply(get_char)
missmatch_seq_test = test_df['MISMATCHED_SEQ'].apply(get_char)
test_y = np.array(test_df['EFFICIENCY'])
onehot_x_1_test = seq_to_onehot(unique_seq_test)
onehot_x_2_test = seq_to_onehot(missmatch_seq_test)
test_x = np.concatenate([onehot_x_1_test, onehot_x_2_test], axis=1)
test_x.shape

(79, 160)

In [11]:
output_path = './miss_match/leav_seq_out'

In [12]:
other_params = {'subsample': 1.0,'min_child_weight': 10, 'gamma': 2.5,'colsample_bytree': 0.4,'alpha': 0, 'learning_rate':0.1}
param_vals = {'n_estimators': [600],'max_depth': [12], 'reg_lambda': [1, 10, 100]}
N_innerCV  =2
inner_cv = KFold(n_splits=N_innerCV, shuffle=True, random_state=0)


In [ ]:
## 4.train
hyper_param = param_vals.values()
combinations = list(itertools.product(*hyper_param))
test_error = []
test_spearm_corr = []
test_pearson_corr = []
error = np.empty([N_innerCV,len(combinations)]) 
## inner cross validation for hyper param tuning
for j, (train_index_new, val_index) in enumerate(inner_cv.split(np.arange(train_x.shape[0]))):
    print(f"Inner Fold {j}:")
    print(f"   number of Training instances:", len(train_index_new))
    print(f"   number of validation instances:", len(val_index))

    for index, c in enumerate(combinations):
        model = xgb.XGBRegressor(n_estimators=c[0], max_depth=c[1], reg_lambda=c[2], **other_params)
        model.fit(train_x[train_index_new,:], train_y[train_index_new], eval_set = [(train_x[val_index,:], train_y[val_index])], early_stopping_rounds=10) 
        ## train error:
        yhat_train = model.predict(train_x[train_index_new,:])
        train_error = np.absolute(train_y[train_index_new]-yhat_train).mean()
        #print('training error:', train_error)
        yhat_val = model.predict(train_x[val_index,:])
        #error = MAE(y[val_index], yhat)
        val_error = np.absolute(train_y[val_index]-yhat_val).mean()
        #print('validation error:', val_error)
        error[j,index] =val_error

#get the best paramerter values       
best_param_indx = error.mean(axis = 0).argmin()
best_param = combinations[best_param_indx]
#best_param = [100, 12, 10]
print('best parameters:', best_param)
#train the model with the best parameters
model = xgb.XGBRegressor(n_estimators=best_param[0], max_depth=best_param[1], reg_lambda=best_param[2], **other_params)
#predict on the test set 
model.fit(train_x, train_y)
yhat_test = model.predict(test_x)
t_error = np.absolute(test_y-yhat_test).mean()
## creat directory to save the model
path_to_save = create_directory(test_seq, output_path)
## save the best model for this run
model.save_model(path_to_save +'/best.model')
ReaderWriter.write_csv(path_to_save+'/prediction', test_y,yhat_test, header=['true','predicted'])
#ReaderWriter.write_log()
test_error.append(t_error)
test_spearm_corr.append(compute_spearman_corr(yhat_test,test_y))
test_pearson_corr.append(compute_pearson_corr(yhat_test,test_y))
perfmetric_report_cont(yhat_test, test_y, path_to_save +'/test_performance.csv' )
ReaderWriter.write_params(path_to_save,param_vals,  best_param, other_params)
fig, ax = plt.subplots()
ax.scatter(yhat_test,test_y,  vmin=0, vmax=100)





In [19]:
test_spearm_corr

[SignificanceResult(statistic=0.15359560910440212, pvalue=0.17654737748471333)]

In [20]:
test_pearson_corr

[PearsonRResult(statistic=0.30947703540451854, pvalue=0.005514533953972714)]

In [21]:
test_error

[24.147825127831045]